# Encontro 13: Medidas de Centralidade

Importando a biblioteca:

In [1]:
import sys
sys.path.append('..')

from random import choice
from itertools import permutations

import pandas as pd
import networkx as nx

import socnet as sn

Configurando a biblioteca:

In [2]:
sn.node_size = 10
sn.node_color = (255, 255, 255)

sn.edge_width = 1
sn.edge_color = (192, 192, 192)

sn.node_label_position = 'top center'

Carregando rede de casamentos entre famílias de Florença durante a Renascença.

J. F. Padgett e C. K. Ansell. *Robust action and the rise of the Medici, 1400–1434.* American Journal of
Sociology 98, págs. 1259-1319, 1993.

In [3]:
g = sn.load_graph('Renaissance.gml', has_pos=True)

sn.show_graph(g, nlab=True)

Função que registra, em cada nó, seus sucessores em geodésicas de $s$ a $t$.

In [4]:
def set_geodesic_successors(g, s, t):
    for n in g.nodes:
        g.nodes[n]['geodesic_successors'] = set()

    for p in nx.all_shortest_paths(g, s, t):
        for i in range(len(p) - 1):
            g.nodes[p[i]]['geodesic_successors'].add(p[i + 1])

Funções que representam uma escolha aleatória de sucessor para diferentes tipos de trajetórias.

In [5]:
# Pense que o atributo 'passages' abaixo indica quantas
# vezes um fluxo já passou por um nó ou por uma aresta.

def random_geodesic_successor(g, n):
    return choice([m for m in g.nodes[n]['geodesic_successors']])

def random_path_successor(g, n):
    return choice([m for m in g.neighbors(n) if g.nodes[m]['passages'] == 0])

def random_trail_successor(g, n):
    return choice([m for m in g.neighbors(n) if g.edges[n, m]['passages'] == 0])

def random_walk_successor(g, n):
    return choice([m for m in g.neighbors(n)])

Função que faz uma simulação de fluxo de $s$ a $t$, que pode ou não ser bem-sucedida.

In [6]:
def simulate_single_flow(g, s, t, whichSuccessorGo, shouldKeep):
    # Inicializa o atributo 'passages' de cada nó.
    for n in g.nodes:
        g.nodes[n]['passages'] = 0
        
    g.nodes[s]['passages'] = 1

    # Inicializa o atributo 'passages' de cada aresta.
    for n, m in g.edges:
        g.edges[n, m]['passages'] = 0

    # Inicializa s como o único dono do insumo.
    for n in g.nodes:
        g.nodes[n]['owner'] = False
    g.nodes[s]['owner'] = True

    # Simula o fluxo, contando o número total de passos.

    steps = 0
    
    funcs = [random_geodesic_successor,random_path_successor,random_trail_successor, random_walk_successor ]

    while True:
        # O conjunto reached representa todos os nós
        # que o fluxo consegue alcançar no passo atual.
        reached = set()

        # Verifica cada um dos donos atuais do insumo.

        owners = [n for n in g.nodes if g.nodes[n]['owner']]

        for n in owners:
            # Deixa de ser dono do insumo.
            if(not shouldKeep):
                g.nodes[n]['owner'] = False

            # Escolhe aleatoriamente um dos sucessores.
            try:
                m = funcs[whichSuccessorGo-1](g, n)
            except:
                continue
                

            # Incrementa o atributo 'passages' do nó.
            g.nodes[m]['passages'] += 1

            # Incrementa o atributo 'passages' da aresta.
            g.edges[n, m]['passages'] += 1

            # Registra que consegue alcançar esse nó.
            reached.add(m)

        # Todo nó alcançado passa a ser dono do insumo.

        for n in reached:
            g.nodes[n]['owner'] = True

        # Isso conclui o passo atual da simulação.
        steps += 1

        # Se o passo alcançou t, chegamos ao fim da simulação.
        # Ela foi bem-sucedida: devolvemos o número de passos.
        if t in reached:
            return steps

        # Se o passo não alcançou ninguém, chegamos ao fim da
        # simulação. Ela não foi bem-sucedida: devolvemos -1.
        if not reached:
            return -1

Função que faz simulações de fluxo de $s$ a $t$ até uma ser bem-sucedida.

In [7]:
def simulate_successful_flow(g, s, t, whichSuccessorGo, shouldKeep):
    set_geodesic_successors(g, s, t)

    while True:
        steps = simulate_single_flow(g, s, t, whichSuccessorGo, shouldKeep)

        if steps != -1:
            return steps

Função que faz simulações de fluxo para todo $s$ e $t$ possíveis, e tira disso um *closeness simulado* e um *betweenness simulado*.

In [8]:
def simulate_all_flows(g,whichSuccessorGo, shouldKeep):
    for n in g.nodes:
        g.nodes[n]['closeness'] = 0
        g.nodes[n]['betweenness'] = 0

    for s, t in permutations(g.nodes, 2):
        steps = simulate_successful_flow(g, s, t, whichSuccessorGo, shouldKeep)

        g.nodes[s]['closeness'] += steps
        for n in g.nodes:
            if n != s and n != t:
                g.nodes[n]['betweenness'] += g.nodes[n]['passages']

    # Normalizações necessárias para comparar com os
    # resultados analíticos. Não é preciso entender.
    for n in g.nodes:
        g.nodes[n]['closeness'] = (g.number_of_nodes() - 1) / g.nodes[n]['closeness']
        g.nodes[n]['betweenness'] /= (g.number_of_nodes() - 1) * (g.number_of_nodes() - 2)

Cálculo de *closeness* e *betweenness* a partir das funções prontas da NetworkX, para comparação.

In [9]:
cc = nx.closeness_centrality(g)

bc = nx.betweenness_centrality(g)

E agora, vamos pensar um pouco...

* Onde você precisa mudar o código para usar uma *trajetória* que não seja a *geodésica*? (caminho, trilha, passeio)

* Onde você precisa mudar o código para usar uma *difusão* que não seja a *transferência*? (duplicação)

Considere então a seguinte **hipótese**:

>Quando consideramos outros tipos de trajetória e outros tipos de difusão, os nós com maior *closeness simulado* e *betweenness simulado* não são necessariamente os nós com maior *closeness* e *betweenness* segundo as fórmulas clássicas. (que correspondem ao uso de geodésica e transferência na simulação)

Queremos:

1. Operacionalização e teste dessas hipótese. (Objetivo 3)
2. Interpretação dos resultados na linguagem de Análise de Redes Sociais (Objetivo 4)

Um *feedback* da atividade sobre *coreness no Jazz* será dado em breve, para vocês terem uma melhor referência do item 2.

In [10]:
def simulate(g,whichFunc, shouldKeep):
    TIMES = 100


    for n in g.nodes:
        g.nodes[n]['mean_closeness'] = 0
        g.nodes[n]['mean_betweenness'] = 0

    for _ in range(TIMES):
        simulate_all_flows(g, whichFunc, shouldKeep)

        for n in g.nodes:
            g.nodes[n]['mean_closeness'] += g.nodes[n]['closeness']
            g.nodes[n]['mean_betweenness'] += g.nodes[n]['betweenness']

    for n in g.nodes:
        g.nodes[n]['mean_closeness'] /= TIMES
        g.nodes[n]['mean_betweenness'] /= TIMES

    return pd.DataFrame({
        'família': [g.nodes[n]['label'] for n in g.nodes],
        'closeness simulado': [g.nodes[n]['mean_closeness'] for n in g.nodes],
        'closeness analítico': [cc[n] for n in g.nodes],
        'betweenness simulado': [g.nodes[n]['mean_betweenness'] for n in g.nodes],
        'betweenness analítico': [bc[n] for n in g.nodes],
    })
    

In [11]:
a = simulate(g,1, True)
a

,betweenness analítico,betweenness simulado,closeness analítico,closeness simulado,família
0,0.000000,0.000000,0.333333,0.333333,ginori
1,0.000000,0.000000,0.325581,0.325581,lambertes
2,0.212454,0.570385,0.482759,0.482759,albizzi
3,0.260073,0.688132,0.466667,0.466667,guadagni
4,0.000000,0.000000,0.285714,0.285714,pazzi
5,0.142857,0.406593,0.388889,0.388889,salviati
6,0.521978,1.390824,0.560000,0.560000,medici
7,0.091575,0.246374,0.482759,0.482759,tornabuon
8,0.120879,0.311978,0.400000,0.400000,bischeri
9,0.086081,0.251538,0.482759,0.482759,ridolfi


In [12]:
b = simulate(g,1, False)
b

,betweenness analítico,betweenness simulado,closeness analítico,closeness simulado,família
0,0.000000,0.000000,0.333333,0.333333,ginori
1,0.000000,0.000000,0.325581,0.325581,lambertes
2,0.212454,0.209286,0.482759,0.482759,albizzi
3,0.260073,0.254231,0.466667,0.466667,guadagni
4,0.000000,0.000000,0.285714,0.285714,pazzi
5,0.142857,0.142857,0.388889,0.388889,salviati
6,0.521978,0.523242,0.560000,0.560000,medici
7,0.091575,0.090110,0.482759,0.482759,tornabuon
8,0.120879,0.120989,0.400000,0.400000,bischeri
9,0.086081,0.091703,0.482759,0.482759,ridolfi


In [13]:
c = simulate(g,2, True)
c

,betweenness analítico,betweenness simulado,closeness analítico,closeness simulado,família
0,0.000000,0.393956,0.333333,0.230717,ginori
1,0.000000,0.402857,0.325581,0.232360,lambertes
2,0.212454,0.616648,0.482759,0.260352,albizzi
3,0.260073,0.742198,0.466667,0.261725,guadagni
4,0.000000,0.262747,0.285714,0.204169,pazzi
5,0.142857,0.437692,0.388889,0.223677,salviati
6,0.521978,0.823462,0.560000,0.260408,medici
7,0.091575,0.682582,0.482759,0.275649,tornabuon
8,0.120879,0.650769,0.400000,0.264431,bischeri
9,0.086081,0.701868,0.482759,0.277214,ridolfi


In [14]:
d = simulate(g,2, False)
d

,betweenness analítico,betweenness simulado,closeness analítico,closeness simulado,família
0,0.000000,0.000000,0.333333,0.206473,ginori
1,0.000000,0.000000,0.325581,0.213169,lambertes
2,0.212454,0.230824,0.482759,0.261651,albizzi
3,0.260073,0.407527,0.466667,0.272935,guadagni
4,0.000000,0.000000,0.285714,0.200320,pazzi
5,0.142857,0.142857,0.388889,0.245080,salviati
6,0.521978,0.626044,0.560000,0.300209,medici
7,0.091575,0.336319,0.482759,0.253408,tornabuon
8,0.120879,0.366813,0.400000,0.252371,bischeri
9,0.086081,0.365604,0.482759,0.255260,ridolfi


In [15]:
e = simulate(g,3, True)
e

,betweenness analítico,betweenness simulado,closeness analítico,closeness simulado,família
0,0.000000,0.367198,0.333333,0.223330,ginori
1,0.000000,0.397418,0.325581,0.225785,lambertes
2,0.212454,0.681593,0.482759,0.247924,albizzi
3,0.260073,0.902253,0.466667,0.253937,guadagni
4,0.000000,0.264451,0.285714,0.199370,pazzi
5,0.142857,0.425110,0.388889,0.221259,salviati
6,0.521978,1.253022,0.560000,0.252287,medici
7,0.091575,0.924890,0.482759,0.267403,tornabuon
8,0.120879,0.785385,0.400000,0.254421,bischeri
9,0.086081,0.929505,0.482759,0.261299,ridolfi


In [16]:
f = simulate(g,3, False)
f

,betweenness analítico,betweenness simulado,closeness analítico,closeness simulado,família
0,0.000000,0.000000,0.333333,0.217085,ginori
1,0.000000,0.000000,0.325581,0.207886,lambertes
2,0.212454,0.261538,0.482759,0.269548,albizzi
3,0.260073,0.407418,0.466667,0.263999,guadagni
4,0.000000,0.000000,0.285714,0.200422,pazzi
5,0.142857,0.142857,0.388889,0.244411,salviati
6,0.521978,0.699341,0.560000,0.276922,medici
7,0.091575,0.321319,0.482759,0.253043,tornabuon
8,0.120879,0.344670,0.400000,0.235004,bischeri
9,0.086081,0.329505,0.482759,0.242994,ridolfi


In [17]:
h = simulate(g,4, True)
h

,betweenness analítico,betweenness simulado,closeness analítico,closeness simulado,família
0,0.000000,0.804066,0.333333,0.146411,ginori
1,0.000000,0.688022,0.325581,0.144658,lambertes
2,0.212454,3.013022,0.482759,0.160735,albizzi
3,0.260073,4.436484,0.466667,0.162046,guadagni
4,0.000000,0.770604,0.285714,0.128214,pazzi
5,0.142857,1.812747,0.388889,0.141634,salviati
6,0.521978,6.910000,0.560000,0.176479,medici
7,0.091575,2.402363,0.482759,0.178015,tornabuon
8,0.120879,2.742418,0.400000,0.158434,bischeri
9,0.086081,2.499121,0.482759,0.167528,ridolfi


In [18]:
t = simulate(g,4, False)
t

,betweenness analítico,betweenness simulado,closeness analítico,closeness simulado,família
0,0.000000,0.742967,0.333333,0.038458,ginori
1,0.000000,0.729286,0.325581,0.038255,lambertes
2,0.212454,2.438736,0.482759,0.039268,albizzi
3,0.260073,3.248132,0.466667,0.036160,guadagni
4,0.000000,0.692912,0.285714,0.039188,pazzi
5,0.142857,1.525165,0.388889,0.037698,salviati
6,0.521978,4.949011,0.560000,0.035305,medici
7,0.091575,2.432582,0.482759,0.037673,tornabuon
8,0.120879,2.420989,0.400000,0.035285,bischeri
9,0.086081,2.413132,0.482759,0.033774,ridolfi


## Analise de erro entre Simulações e Analíticas

In [19]:
import numpy as np
import matplotlib.pyplot as plt

In [47]:
def getErrors(df):
    result_b = np.abs(df["betweenness analítico"] - df["betweenness simulado"])
    result_b = np.mean(result_b)
    result_c =  np.abs(df["closeness analítico"] - df["closeness simulado"])
    result_c =  np.mean(result_c)
    print("Média de erro entre simulado e analítico (betweenness: {0})".format(result_b))
    print("Média de erro entre simulado e analítico (closeness: {0})".format(result_c))
    print("\n")
    return result_b,result_c

In [48]:
getErrors(a)
getErrors(b)
getErrors(c)
getErrors(d)
getErrors(e)
getErrors(f)
getErrors(h)
getErrors(t)

Média de erro entre simulado e analítico (betweenness: 0.19614652014652006)
Média de erro entre simulado e analítico (closeness: 4.51490696680897e-16)


Média de erro entre simulado e analítico (betweenness: 0.0015506715506715596)
Média de erro entre simulado e analítico (closeness: 4.51490696680897e-16)


Média de erro entre simulado e analítico (betweenness: 0.4524578754578755)
Média de erro entre simulado e analítico (closeness: 0.16062475770836288)


Média de erro entre simulado e analítico (betweenness: 0.13213186813186806)
Média de erro entre simulado e analítico (closeness: 0.16862872886977107)


Média de erro entre simulado e analítico (betweenness: 0.5641282051282049)
Média de erro entre simulado e analítico (closeness: 0.17032473460804834)


Média de erro entre simulado e analítico (betweenness: 0.13096703296703296)
Média de erro entre simulado e analítico (closeness: 0.1717425992605356)


Média de erro entre simulado e analítico (betweenness: 2.252893772893773)
Média de erro

(1.908648351648352, 0.3747415519689021)

## Análise de intensidade de importância para diferentes funções (betweenness)

In [49]:
print('Geodésicas clonando: ')
print(np.mean(a["betweenness simulado"]))
print('Geodésicas transferindo: ')
print(np.mean(b["betweenness simulado"]))
print("\n")

print('Caminho clonando: ')
print(np.mean(c["betweenness simulado"]))
print('Caminho transferindo: ')
print(np.mean(d["betweenness simulado"]))
print("\n")

print('Trilha clonando: ')
print(np.mean(e["betweenness simulado"]))
print('Trilha transferindo: ')
print(np.mean(f["betweenness simulado"]))
print("\n")

print('Passeio clonando: ')
print(np.mean(h["betweenness simulado"]))
print('Passeio transferindo: ')
print(np.mean(t["betweenness simulado"]))
print("\n")

Geodésicas clonando: 
0.3118974358974358
Geodésicas transferindo: 
0.11575091575091569


Caminho clonando: 
0.5682087912087912
Caminho transferindo: 
0.24788278388278379


Trilha clonando: 
0.6798791208791207
Trilha transferindo: 
0.24671794871794867


Passeio clonando: 
2.3686446886446886
Passeio transferindo: 
2.0243992673992675




## Análise de ordem importância para diferentes funções

Por enquanto analisando esses resultados percebe-se que manter o pacote influenciou mais na disparidade dos resultados em relação ao betweness do que ao closeness. Vamos analisar agora se a sequência de "importância" mantêm-se.

In [24]:
def generateAnalisys(a):

    listaBA = a["betweenness analítico"]
    sortedListBA = []
    for i in range(0,len(listaBA)):
        sortedListBA.append((i,listaBA[i]))

    sortedListBA = sorted(sortedListBA, key=lambda x: x[1], reverse=True)
    #print(sortedListBA)
    
    listaBS = a["betweenness simulado"]

    sortedListBS = []
    for i in range(0,len(listaBS)):
        sortedListBS.append((i,listaBS[i]))

    sortedListBS = sorted(sortedListBS, key=lambda x: x[1], reverse=True)
    #print(sortedListBS)

    listaCA = a["closeness analítico"]
    sortedListCA = []
    for i in range(0,len(listaCA)):
        sortedListCA.append((i,listaCA[i]))

    sortedListCA = sorted(sortedListCA, key=lambda x: x[1], reverse=True)
    #print(sortedListCA)

    listaCS = a["closeness simulado"]
    sortedListCS = []
    for i in range(0,len(listaCS)):
        sortedListCS.append((i,listaCS[i]))

    sortedListCS = sorted(sortedListCS, key=lambda x: x[1], reverse=True)
    #print(sortedListCS)
    
    return(sortedListBA, sortedListBS, sortedListCA, sortedListCS)

In [25]:
BAa, BSa, CAa, CSa = generateAnalisys(a)
print("Betweenness Analítico")
print(BAa)
print("\n")
print("Betweenness Simulado")
print(BSa)
print("\n")
print("Closeness Analítico")
print(CAa)
print("\n")
print("Closeness Simulado")
print(CSa)


Betweenness Analítico
[(6, 0.521978021978022), (3, 0.2600732600732601), (2, 0.21245421245421245), (5, 0.14285714285714288), (8, 0.1208791208791209), (13, 0.11538461538461539), (7, 0.09157509157509161), (14, 0.08791208791208792), (9, 0.08608058608058608), (11, 0.07509157509157509), (12, 0.02197802197802198), (0, 0.0), (1, 0.0), (4, 0.0), (10, 0.0)]


Betweenness Simulado
[(6, 1.3908241758241748), (3, 0.6881318681318688), (2, 0.5703846153846158), (5, 0.4065934065934066), (13, 0.31747252747252747), (8, 0.31197802197802205), (9, 0.2515384615384615), (7, 0.2463736263736263), (14, 0.2325824175824173), (11, 0.2069780219780218), (12, 0.055604395604395535), (0, 0.0), (1, 0.0), (4, 0.0), (10, 0.0)]


Closeness Analítico
[(6, 0.56), (2, 0.4827586206896552), (7, 0.4827586206896552), (9, 0.4827586206896552), (3, 0.4666666666666667), (13, 0.4375), (11, 0.42424242424242425), (8, 0.4), (5, 0.3888888888888889), (14, 0.3888888888888889), (10, 0.3684210526315789), (12, 0.35), (0, 0.3333333333333333), (1,

In [26]:
BAb, BSb, CAb, CSb = generateAnalisys(b)
print("Betweenness Analítico")
print(BAb)
print("\n")
print("Betweenness Simulado")
print(BSb)
print("\n")
print("Closeness Analítico")
print(CAb)
print("\n")
print("Closeness Simulado")
print(CSb)

Betweenness Analítico
[(6, 0.521978021978022), (3, 0.2600732600732601), (2, 0.21245421245421245), (5, 0.14285714285714288), (8, 0.1208791208791209), (13, 0.11538461538461539), (7, 0.09157509157509161), (14, 0.08791208791208792), (9, 0.08608058608058608), (11, 0.07509157509157509), (12, 0.02197802197802198), (0, 0.0), (1, 0.0), (4, 0.0), (10, 0.0)]


Betweenness Simulado
[(6, 0.5232417582417586), (3, 0.25423076923076926), (2, 0.20928571428571413), (5, 0.1428571428571427), (8, 0.12098901098901076), (13, 0.11543956043956019), (9, 0.09170329670329655), (7, 0.09010989010988996), (14, 0.08675824175824162), (11, 0.079120879120879), (12, 0.022527472527472506), (0, 0.0), (1, 0.0), (4, 0.0), (10, 0.0)]


Closeness Analítico
[(6, 0.56), (2, 0.4827586206896552), (7, 0.4827586206896552), (9, 0.4827586206896552), (3, 0.4666666666666667), (13, 0.4375), (11, 0.42424242424242425), (8, 0.4), (5, 0.3888888888888889), (14, 0.3888888888888889), (10, 0.3684210526315789), (12, 0.35), (0, 0.3333333333333333),

In [27]:
BAc, BSc, CAc, CSc = generateAnalisys(c)
print("Betweenness Analítico")
print(BAc)
print("\n")
print("Betweenness Simulado")
print(BSc)
print("\n")
print("Closeness Analítico")
print(CAc)
print("\n")
print("Closeness Simulado")
print(CSc)

Betweenness Analítico
[(6, 0.521978021978022), (3, 0.2600732600732601), (2, 0.21245421245421245), (5, 0.14285714285714288), (8, 0.1208791208791209), (13, 0.11538461538461539), (7, 0.09157509157509161), (14, 0.08791208791208792), (9, 0.08608058608058608), (11, 0.07509157509157509), (12, 0.02197802197802198), (0, 0.0), (1, 0.0), (4, 0.0), (10, 0.0)]


Betweenness Simulado
[(6, 0.8234615384615384), (3, 0.7421978021978022), (9, 0.701868131868132), (7, 0.6825824175824181), (11, 0.6814835164835165), (8, 0.650769230769231), (14, 0.6174725274725276), (2, 0.6166483516483519), (13, 0.5849450549450553), (12, 0.5269780219780222), (5, 0.43769230769230755), (1, 0.40285714285714264), (10, 0.39747252747252715), (0, 0.3939560439560437), (4, 0.2627472527472528)]


Closeness Analítico
[(6, 0.56), (2, 0.4827586206896552), (7, 0.4827586206896552), (9, 0.4827586206896552), (3, 0.4666666666666667), (13, 0.4375), (11, 0.42424242424242425), (8, 0.4), (5, 0.3888888888888889), (14, 0.3888888888888889), (10, 0.36

In [28]:
BAd, BSd, CAd, CSd = generateAnalisys(d)
print("Betweenness Analítico")
print(BAd)
print("\n")
print("Betweenness Simulado")
print(BSd)
print("\n")
print("Closeness Analítico")
print(CAd)
print("\n")
print("Closeness Simulado")
print(CSd)

Betweenness Analítico
[(6, 0.521978021978022), (3, 0.2600732600732601), (2, 0.21245421245421245), (5, 0.14285714285714288), (8, 0.1208791208791209), (13, 0.11538461538461539), (7, 0.09157509157509161), (14, 0.08791208791208792), (9, 0.08608058608058608), (11, 0.07509157509157509), (12, 0.02197802197802198), (0, 0.0), (1, 0.0), (4, 0.0), (10, 0.0)]


Betweenness Simulado
[(6, 0.6260439560439562), (3, 0.4075274725274722), (14, 0.3836813186813185), (11, 0.3677472527472526), (8, 0.3668131868131866), (9, 0.3656043956043954), (7, 0.33631868131868137), (12, 0.2513736263736265), (13, 0.23945054945054944), (2, 0.23082417582417558), (5, 0.1428571428571427), (0, 0.0), (1, 0.0), (4, 0.0), (10, 0.0)]


Closeness Analítico
[(6, 0.56), (2, 0.4827586206896552), (7, 0.4827586206896552), (9, 0.4827586206896552), (3, 0.4666666666666667), (13, 0.4375), (11, 0.42424242424242425), (8, 0.4), (5, 0.3888888888888889), (14, 0.3888888888888889), (10, 0.3684210526315789), (12, 0.35), (0, 0.3333333333333333), (1, 

In [29]:
BAe, BSe, CAe, CSe = generateAnalisys(e)
print("Betweenness Analítico")
print(BAe)
print("\n")
print("Betweenness Simulado")
print(BSe)
print("\n")
print("Closeness Analítico")
print(CAe)
print("\n")
print("Closeness Simulado")
print(CSe)

Betweenness Analítico
[(6, 0.521978021978022), (3, 0.2600732600732601), (2, 0.21245421245421245), (5, 0.14285714285714288), (8, 0.1208791208791209), (13, 0.11538461538461539), (7, 0.09157509157509161), (14, 0.08791208791208792), (9, 0.08608058608058608), (11, 0.07509157509157509), (12, 0.02197802197802198), (0, 0.0), (1, 0.0), (4, 0.0), (10, 0.0)]


Betweenness Simulado
[(6, 1.2530219780219778), (9, 0.9295054945054945), (7, 0.9248901098901094), (3, 0.9022527472527466), (11, 0.8928021978021974), (14, 0.8199450549450547), (8, 0.7853846153846153), (2, 0.6815934065934067), (13, 0.5922527472527478), (12, 0.5910989010989014), (5, 0.4251098901098898), (1, 0.3974175824175821), (10, 0.37126373626373604), (0, 0.36719780219780207), (4, 0.26445054945054947)]


Closeness Analítico
[(6, 0.56), (2, 0.4827586206896552), (7, 0.4827586206896552), (9, 0.4827586206896552), (3, 0.4666666666666667), (13, 0.4375), (11, 0.42424242424242425), (8, 0.4), (5, 0.3888888888888889), (14, 0.3888888888888889), (10, 0.

In [30]:
BAf, BSf, CAf, CSf = generateAnalisys(f)
print("Betweenness Analítico")
print(BAf)
print("\n")
print("Betweenness Simulado")
print(BSf)
print("\n")
print("Closeness Analítico")
print(CAf)
print("\n")
print("Closeness Simulado")
print(CSf)

Betweenness Analítico
[(6, 0.521978021978022), (3, 0.2600732600732601), (2, 0.21245421245421245), (5, 0.14285714285714288), (8, 0.1208791208791209), (13, 0.11538461538461539), (7, 0.09157509157509161), (14, 0.08791208791208792), (9, 0.08608058608058608), (11, 0.07509157509157509), (12, 0.02197802197802198), (0, 0.0), (1, 0.0), (4, 0.0), (10, 0.0)]


Betweenness Simulado
[(6, 0.6993406593406594), (3, 0.40741758241758214), (14, 0.37219780219780196), (8, 0.34467032967032957), (9, 0.3295054945054946), (11, 0.32417582417582413), (7, 0.3213186813186814), (13, 0.26989010989011), (2, 0.2615384615384617), (12, 0.22785714285714267), (5, 0.1428571428571427), (0, 0.0), (1, 0.0), (4, 0.0), (10, 0.0)]


Closeness Analítico
[(6, 0.56), (2, 0.4827586206896552), (7, 0.4827586206896552), (9, 0.4827586206896552), (3, 0.4666666666666667), (13, 0.4375), (11, 0.42424242424242425), (8, 0.4), (5, 0.3888888888888889), (14, 0.3888888888888889), (10, 0.3684210526315789), (12, 0.35), (0, 0.3333333333333333), (1, 

In [31]:
BAh, BSh, CAh, CSh = generateAnalisys(h)
print("Betweenness Analítico")
print(BAh)
print("\n")
print("Betweenness Simulado")
print(BSh)
print("\n")
print("Closeness Analítico")
print(CAh)
print("\n")
print("Closeness Simulado")
print(CSh)

Betweenness Analítico
[(6, 0.521978021978022), (3, 0.2600732600732601), (2, 0.21245421245421245), (5, 0.14285714285714288), (8, 0.1208791208791209), (13, 0.11538461538461539), (7, 0.09157509157509161), (14, 0.08791208791208792), (9, 0.08608058608058608), (11, 0.07509157509157509), (12, 0.02197802197802198), (0, 0.0), (1, 0.0), (4, 0.0), (10, 0.0)]


Betweenness Simulado
[(6, 6.910000000000002), (3, 4.436483516483515), (14, 3.081703296703297), (2, 3.013021978021978), (8, 2.742417582417582), (11, 2.6851098901098895), (9, 2.499120879120879), (7, 2.4023626373626383), (5, 1.8127472527472535), (12, 1.6445054945054942), (13, 1.456813186813186), (0, 0.8040659340659337), (4, 0.7706043956043955), (1, 0.6880219780219778), (10, 0.582692307692308)]


Closeness Analítico
[(6, 0.56), (2, 0.4827586206896552), (7, 0.4827586206896552), (9, 0.4827586206896552), (3, 0.4666666666666667), (13, 0.4375), (11, 0.42424242424242425), (8, 0.4), (5, 0.3888888888888889), (14, 0.3888888888888889), (10, 0.36842105263

In [32]:
BAt, BSt, CAt, CSt = generateAnalisys(t)
print("Betweenness Analítico")
print(BAt)
print("\n")
print("Betweenness Simulado")
print(BSt)
print("\n")
print("Closeness Analítico")
print(CAt)
print("\n")
print("Closeness Simulado")
print(CSt)

Betweenness Analítico
[(6, 0.521978021978022), (3, 0.2600732600732601), (2, 0.21245421245421245), (5, 0.14285714285714288), (8, 0.1208791208791209), (13, 0.11538461538461539), (7, 0.09157509157509161), (14, 0.08791208791208792), (9, 0.08608058608058608), (11, 0.07509157509157509), (12, 0.02197802197802198), (0, 0.0), (1, 0.0), (4, 0.0), (10, 0.0)]


Betweenness Simulado
[(6, 4.949010989010988), (3, 3.248131868131868), (2, 2.4387362637362635), (14, 2.4343406593406596), (7, 2.4325824175824184), (8, 2.4209890109890106), (11, 2.4186813186813194), (9, 2.4131318681318676), (13, 1.5986263736263737), (12, 1.5787362637362636), (5, 1.5251648351648357), (0, 0.7429670329670334), (10, 0.7426923076923075), (1, 0.7292857142857142), (4, 0.6929120879120879)]


Closeness Analítico
[(6, 0.56), (2, 0.4827586206896552), (7, 0.4827586206896552), (9, 0.4827586206896552), (3, 0.4666666666666667), (13, 0.4375), (11, 0.42424242424242425), (8, 0.4), (5, 0.3888888888888889), (14, 0.3888888888888889), (10, 0.36842

Vamos dar uma verificada no comportamento desses testes para diferentes posições, analisando as extremidades, quando se tratando de betweenness.

In [36]:
allListsB = [BSa, BSb,  BSc, BSd, BSe, BSf,  BSh,  BSt]

In [39]:
for i in allListsB:
    print('Primeiro elemento é o nó: {0}'.format(i[0][0]))
    print('Segundo elemento é o nó: {0}'.format(i[1][0]))
    print('Penultimo elemento é o nó: {0}'.format(i[len(i)-2][0]))
    print('Ultimo elemento é o nó: {0}'.format(i[len(i)-1][0]))
    print('\n')

Primeiro elemento é o nó: 6
Segundo elemento é o nó: 3
Penultimo elemento é o nó: 4
Ultimo elemento é o nó: 10


Primeiro elemento é o nó: 6
Segundo elemento é o nó: 3
Penultimo elemento é o nó: 4
Ultimo elemento é o nó: 10


Primeiro elemento é o nó: 6
Segundo elemento é o nó: 3
Penultimo elemento é o nó: 0
Ultimo elemento é o nó: 4


Primeiro elemento é o nó: 6
Segundo elemento é o nó: 3
Penultimo elemento é o nó: 4
Ultimo elemento é o nó: 10


Primeiro elemento é o nó: 6
Segundo elemento é o nó: 9
Penultimo elemento é o nó: 0
Ultimo elemento é o nó: 4


Primeiro elemento é o nó: 6
Segundo elemento é o nó: 3
Penultimo elemento é o nó: 4
Ultimo elemento é o nó: 10


Primeiro elemento é o nó: 6
Segundo elemento é o nó: 3
Penultimo elemento é o nó: 1
Ultimo elemento é o nó: 10


Primeiro elemento é o nó: 6
Segundo elemento é o nó: 3
Penultimo elemento é o nó: 1
Ultimo elemento é o nó: 4




In [40]:
allListsC = [CSa, CSb,  CSc, CSd, CSe, CSf,  CSh,  CSt]

In [41]:
for i in allListsC:
    print('Primeiro elemento é o nó: {0}'.format(i[0][0]))
    print('Segundo elemento é o nó: {0}'.format(i[1][0]))
    print('Penultimo elemento é o nó: {0}'.format(i[len(i)-2][0]))
    print('Ultimo elemento é o nó: {0}'.format(i[len(i)-1][0]))
    print('\n')

Primeiro elemento é o nó: 6
Segundo elemento é o nó: 2
Penultimo elemento é o nó: 1
Ultimo elemento é o nó: 4


Primeiro elemento é o nó: 6
Segundo elemento é o nó: 2
Penultimo elemento é o nó: 1
Ultimo elemento é o nó: 4


Primeiro elemento é o nó: 9
Segundo elemento é o nó: 7
Penultimo elemento é o nó: 5
Ultimo elemento é o nó: 4


Primeiro elemento é o nó: 6
Segundo elemento é o nó: 3
Penultimo elemento é o nó: 0
Ultimo elemento é o nó: 4


Primeiro elemento é o nó: 7
Segundo elemento é o nó: 9
Penultimo elemento é o nó: 5
Ultimo elemento é o nó: 4


Primeiro elemento é o nó: 6
Segundo elemento é o nó: 2
Penultimo elemento é o nó: 1
Ultimo elemento é o nó: 4


Primeiro elemento é o nó: 7
Segundo elemento é o nó: 6
Penultimo elemento é o nó: 5
Ultimo elemento é o nó: 4


Primeiro elemento é o nó: 2
Segundo elemento é o nó: 4
Penultimo elemento é o nó: 13
Ultimo elemento é o nó: 9


